In [1]:
import numpy as np
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *

# Loading your IBM Quantum account(s)
provider = IBMQ.load_account()

In [2]:
import networkx as nx
import numpy as np
import plotly.graph_objects as go
import matplotlib as mpl
import pandas as pd
from IPython.display import clear_output
from plotly.subplots import make_subplots
from matplotlib import pyplot as plt
from qiskit import Aer
from qiskit import QuantumCircuit
from qiskit.visualization import plot_state_city
from qiskit.algorithms.optimizers import COBYLA, SLSQP, ADAM
from time import time
from copy import copy
from typing import List

In [4]:
pip install qiskit_optimization --upgrade

  Using cached qiskit_optimization-0.2.0-py3-none-any.whl (141 kB)
  Using cached networkx-2.5.1-py3-none-any.whl (1.6 MB)
  Using cached docplex-2.21.207-py3-none-any.whl
  Using cached decorator-4.4.2-py2.py3-none-any.whl (9.2 kB)
  Attempting uninstall: decorator
    Found existing installation: decorator 5.0.9
    Uninstalling decorator-5.0.9:
      Successfully uninstalled decorator-5.0.9
  Attempting uninstall: networkx
    Found existing installation: networkx 2.6.1
    Uninstalling networkx-2.6.1:
      Successfully uninstalled networkx-2.6.1
  Attempting uninstall: docplex
    Found existing installation: docplex 2.20.204
    Uninstalling docplex-2.20.204:
      Successfully uninstalled docplex-2.20.204
  Attempting uninstall: qiskit-optimization
    Found existing installation: qiskit-optimization 0.1.0
    Uninstalling qiskit-optimization-0.1.0:
      Successfully uninstalled qiskit-optimization-0.1.0
ERROR: pip's dependency resolver does not currently take into account all 

In [5]:
from typing import Dict, List, Optional, Union

import networkx as nx
import numpy as np
from docplex.mp.model import Model

from qiskit_optimization.algorithms import OptimizationResult
from qiskit_optimization.problems.quadratic_program import QuadraticProgram
from qiskit_optimization.translators import from_docplex_mp
#from qiskit_optimization.graph import GraphOptimizationApplication
from qiskit_optimization.applications.graph_optimization_application import GraphOptimizationApplication

In [6]:
class VehicleRouting(GraphOptimizationApplication):
    def __init__(
        self,
        graph: Union[nx.Graph, np.ndarray, List],
        num_vehicles: int = 2,
        depot: int = 0,
    ) -> None:
        
        
        super().__init__(graph)
        self._num_vehicles = num_vehicles
        self._depot = depot

    def to_quadratic_program(self) -> QuadraticProgram:
        mdl = Model(name="Vehicle routing")
        n = self._graph.number_of_nodes()
        x = {}
        for i in range(n):
            for j in range(n):
                if i != j:
                    x[(i, j)] = mdl.binary_var(name="x_{0}_{1}".format(i, j))
        mdl.minimize(
            mdl.sum(
                self._graph.edges[i, j]["weight"] * x[(i, j)]
                for i in range(n)
                for j in range(n)
                if i != j
            )
        )
        # Only 1 edge goes out from each node
        for i in range(n):
            if i != self.depot:
                mdl.add_constraint(mdl.sum(x[i, j] for j in range(n) if i != j) == 1)
        # Only 1 edge comes into each node
        for j in range(n):
            if j != self.depot:
                mdl.add_constraint(mdl.sum(x[i, j] for i in range(n) if i != j) == 1)
        # For the depot node
        mdl.add_constraint(
            mdl.sum(x[i, self.depot] for i in range(n) if i != self.depot) == self.num_vehicles
        )
        mdl.add_constraint(
            mdl.sum(x[self.depot, j] for j in range(n) if j != self.depot) == self.num_vehicles
        )

        # To eliminate sub-routes
        node_list = [i for i in range(n) if i != self.depot]
        clique_set = []
        for i in range(2, len(node_list) + 1):
            for comb in itertools.combinations(node_list, i):
                clique_set.append(list(comb))
        for clique in clique_set:
            mdl.add_constraint(
                mdl.sum(x[(i, j)] for i in clique for j in clique if i != j) <= len(clique) - 1
            )
        op = from_docplex_mp(mdl)
        return op

    def interpret(self, result: Union[OptimizationResult, np.ndarray]) -> List[List[List[int]]]:
        x = self._result_to_x(result)
        n = self._graph.number_of_nodes()
        idx = 0
        edge_list = []
        for i in range(n):
            for j in range(n):
                if i != j:
                    if x[idx]:
                        edge_list.append([i, j])
                    idx += 1
        route_list = []  # type: List[List[List[int]]]
        for k in range(self.num_vehicles):
            i = 0
            start = self.depot
            route_list.append([])
            while i < len(edge_list):
                if edge_list[i][0] == start:
                    if edge_list[i][1] == self.depot:
                        # If a loop is completed
                        route_list[k].append(edge_list.pop(i))
                        break
                    # Move onto the next edge
                    start = edge_list[i][1]
                    route_list[k].append(edge_list.pop(i))
                    i = 0
                    continue
                i += 1
        if edge_list:
            route_list.append(edge_list)

        return route_list

    def _draw_result(
        self,
        result: Union[OptimizationResult, np.ndarray],
        pos: Optional[Dict[int, np.ndarray]] = None,
    ) -> None:
        import matplotlib.pyplot as plt

        route_list = self.interpret(result)
        nx.draw(self._graph, with_labels=True, pos=pos)
        nx.draw_networkx_edges(
            self._graph,
            pos,
            edgelist=self._edgelist(route_list),
            width=8,
            alpha=0.5,
            edge_color=self._edge_color(route_list),
            edge_cmap=plt.cm.plasma,
        )

    def _edgelist(self, route_list: List[List[List[int]]]):
        # Arrange route_list and return the list of the edges for the edge list of
        # nx.draw_networkx_edges
        return [edge for k in range(len(route_list)) for edge in route_list[k]]

    def _edge_color(self, route_list: List[List[List[int]]]):
        # Arrange route_list and return the list of the colors of each route
        # for edge_color of nx.draw_networkx_edges
        return [k / len(route_list) for k in range(len(route_list)) for edge in route_list[k]]

    @property
    def num_vehicles(self) -> int:
        """Getter of num_vehicles
        Returns:
            The number of the vehicles
        """
        return self._num_vehicles

    @num_vehicles.setter
    def num_vehicles(self, num_vehicles: int) -> None:
        self._num_vehicles = num_vehicles

    @property
    def depot(self) -> int:
        return self._depot

    @depot.setter
    def depot(self, depot: int) -> None:
        self._depot = depot

    @staticmethod
    # pylint: disable=undefined-variable
    def create_random_instance(
        n: int,
        low: int = 0,
        high: int = 100,
        seed: Optional[int] = None,
        num_vehicle: int = 2,
        depot: int = 0,
    ) -> "VehicleRouting":
        random.seed(seed)
        pos = {i: (random.randint(low, high), random.randint(low, high)) for i in range(n)}
        graph = nx.random_geometric_graph(n, np.hypot(high - low, high - low) + 1, pos=pos)
        for w, v in graph.edges:
            delta = [graph.nodes[w]["pos"][i] - graph.nodes[v]["pos"][i] for i in range(2)]
            graph.edges[w, v]["weight"] = np.rint(np.hypot(delta[0], delta[1]))
        return VehicleRouting(graph, num_vehicle, depot)